In [161]:
# Importar as bibliotecas
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [162]:
# Importar dataframe
ticker = 'BOVA11.SA'
dtini = '2016-01-01'
dtfin = '2025-12-31'
df = yf.download(ticker, dtini, dtfin, auto_adjust=False)
df.columns=df.columns.get_level_values(0)

[*********************100%***********************]  1 of 1 completed


In [163]:
# Primeira inspeção dos dados
print(f"Dimensões: {df.shape}")
print(f"\nColunas: {list(df.columns)}")
print(f"\nTipos de dados:")
print(df.dtypes)
print(f"\nInformações gerais:")
print(df.info())
print(f"\nPrimeiras 5 linhas:")
print(df.head())

Dimensões: (2325, 6)

Colunas: ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

Tipos de dados:
Price
Adj Close    float64
Close        float64
High         float64
Low          float64
Open         float64
Volume         int64
dtype: object

Informações gerais:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2325 entries, 2016-01-04 to 2025-06-11
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  2325 non-null   float64
 1   Close      2325 non-null   float64
 2   High       2325 non-null   float64
 3   Low        2325 non-null   float64
 4   Open       2325 non-null   float64
 5   Volume     2325 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 127.1 KB
None

Primeiras 5 linhas:
Price       Adj Close      Close       High        Low       Open   Volume
Date                                                                      
2016-01-04  41.099998  41.099998  42.299999  40.799999  4

In [164]:
# Análise exploratória de dados
print("\nVALORES AUSENTES")
print("-" * 30)
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Coluna': missing.index,
    'Valores_Ausentes': missing.values,
    'Porcentagem': missing_percent.values
})
print(missing_df[missing_df['Valores_Ausentes'] > 0])

print("\nESTATÍSTICAS DESCRITIVAS")
print("-" * 30)
print(df.describe())


VALORES AUSENTES
------------------------------
Empty DataFrame
Columns: [Coluna, Valores_Ausentes, Porcentagem]
Index: []

ESTATÍSTICAS DESCRITIVAS
------------------------------
Price    Adj Close        Close         High          Low         Open  \
count  2325.000000  2325.000000  2325.000000  2325.000000  2325.000000   
mean     95.148469    95.148469    95.917789    94.373054    95.180452   
std      24.104419    24.104419    24.177101    24.040872    24.099613   
min      36.450001    36.450001    36.610001    36.009998    36.320000   
25%      75.269997    75.269997    75.989998    74.300003    75.300003   
50%     100.379997   100.379997   101.150002    99.610001   100.379997   
75%     114.000000   114.000000   114.849998   113.180000   114.000000   
max     136.839996   136.839996   137.190002   136.149994   136.649994   

Price        Volume  
count  2.325000e+03  
mean   5.874308e+06  
std    3.919309e+06  
min    5.864600e+05  
25%    3.227550e+06  
50%    5.028050e+06 

In [165]:
# Análise estatística por coluna
for col in df.columns:
    print(f"\nAnálise da coluna: {col}")
    print(f"Média: {df[col].mean():.2f}")
    print(f"Mediana: {df[col].median():.2f}")
    print(f"Moda: {df[col].mode().iloc[0] if not df[col].mode().empty else 'N/A'}")
    print(f"Desvio Padrão: {df[col].std():.2f}")
    print(f"Assimetria: {df[col].skew():.2f}")
    print(f"Curtose: {df[col].kurtosis():.2f}")
    print(f"Mín: {df[col].min():.2f} | Máx: {df[col].max():.2f}")


Análise da coluna: Adj Close
Média: 95.15
Mediana: 100.38
Moda: 100.0
Desvio Padrão: 24.10
Assimetria: -0.49
Curtose: -0.81
Mín: 36.45 | Máx: 136.84

Análise da coluna: Close
Média: 95.15
Mediana: 100.38
Moda: 100.0
Desvio Padrão: 24.10
Assimetria: -0.49
Curtose: -0.81
Mín: 36.45 | Máx: 136.84

Análise da coluna: High
Média: 95.92
Mediana: 101.15
Moda: 114.9000015258789
Desvio Padrão: 24.18
Assimetria: -0.50
Curtose: -0.80
Mín: 36.61 | Máx: 137.19

Análise da coluna: Low
Média: 94.37
Mediana: 99.61
Moda: 97.41999816894531
Desvio Padrão: 24.04
Assimetria: -0.47
Curtose: -0.81
Mín: 36.01 | Máx: 136.15

Análise da coluna: Open
Média: 95.18
Mediana: 100.38
Moda: 121.5
Desvio Padrão: 24.10
Assimetria: -0.49
Curtose: -0.80
Mín: 36.32 | Máx: 136.65

Análise da coluna: Volume
Média: 5874308.45
Mediana: 5028050.00
Moda: 4069100
Desvio Padrão: 3919308.81
Assimetria: 2.49
Curtose: 12.56
Mín: 586460.00 | Máx: 45849230.00


In [166]:
# Cálculo da diferença do preço e da média
df['MM'] = df['Adj Close'].rolling(20).mean()
df['Delta'] = df['Adj Close'] - df['MM']

In [167]:
# Analisar a série Delta
delta_clean = df['Delta'].dropna()
print("ANÁLISE SÉRIE DELTA")
print("="*60)

# Estatísticas descritivas
print(f"Média: {delta_clean.mean():.4f}")
print(f"Mediana: {delta_clean.median():.4f}")
print(f"Desvio Padrão: {delta_clean.std():.4f}")
print(f"Assimetria (Skewness): {stats.skew(delta_clean):.4f}")
print(f"Curtose (Kurtosis): {stats.kurtosis(delta_clean):.4f}")

ANÁLISE SÉRIE DELTA
Média: 0.3974
Mediana: 0.5130
Desvio Padrão: 3.3229
Assimetria (Skewness): -2.0726
Curtose (Kurtosis): 14.7396


In [168]:
# Definir parâmetros
delta = df['Delta']
janela=20
percentil=2
alvo_dias=10

# Usando percentis móveis
media_movel = delta.rolling(window=janela).median()
banda_sup = delta.rolling(window=janela).quantile(1 - percentil/100)
banda_inf = delta.rolling(window=janela).quantile(percentil/100)

# Adicionar ao DataFrame
df[f'banda_sup'] = banda_sup
df[f'banda_inf'] = banda_inf
df[f'media'] = media_movel

# Detectar anomalias
df['anomalia_superior'] = df['Delta'] > df[f'banda_sup']
df['anomalia_inferior'] = df['Delta'] < df[f'banda_inf']
df['anomalia_qualquer'] = df['anomalia_superior'] | df['anomalia_inferior']

# Calcular retornos e alvos
df['Retorno'] = df['Close'].pct_change(alvo_dias)
df['Alvo'] = df['Retorno'].shift(-alvo_dias)

# Regras de trading
df['Regra'] = np.where((df['anomalia_inferior'])&(df['Delta']>-5), 1, 0)

# Calcular trades e performance
df['Trade'] = df['Alvo'] * df['Regra']
df['Acumulado'] = df['Trade'].cumsum() * 100

# Remover NaN
df = df.dropna()

In [169]:
#Relatório de performance
trades = df[df['Regra'] != 0]['Trade']

print("ANÁLISE DE PERFORMANCE")
print("="*60)

# Estatísticas gerais
print(f"Total de trades: {len(trades)}")
print(f"Trades positivos: {(trades > 0).sum()} ({(trades > 0).mean()*100:.1f}%)")
print(f"Trades negativos: {(trades < 0).sum()} ({(trades < 0).mean()*100:.1f}%)")

# Performance
retorno_total = df['Acumulado'].iloc[-1]
print(f"\nRetorno total: {retorno_total:.2f}%")
print(f"Retorno médio por trade: {trades.mean()*100:.3f}%")
print(f"Desvio padrão dos trades: {trades.std()*100:.3f}%")

# Sharpe ratio simplificado
if trades.std() != 0:
    sharpe = trades.mean() / trades.std()
    print(f"Sharpe ratio: {sharpe:.3f}")

# Drawdown máximo
df['Running_Max'] = df['Acumulado'].expanding().max()
df['Drawdown'] = df['Acumulado'] - df['Running_Max']
max_drawdown = df['Drawdown'].min()
print(f"Drawdown máximo: {max_drawdown:.2f}%")

# Anomalias detectadas
anomalias_sup = df['anomalia_superior'].sum()
anomalias_inf = df['anomalia_inferior'].sum()
total_anomalias = anomalias_sup + anomalias_inf

print(f"\nAnomalias detectadas:")
print(f"  Superiores (venda): {anomalias_sup}")
print(f"  Inferiores (compra): {anomalias_inf}")
print(f"  Total: {total_anomalias}")
print(f"  Frequência: {total_anomalias/len(df)*100:.2f}% dos dias")


ANÁLISE DE PERFORMANCE
Total de trades: 228
Trades positivos: 144 (63.2%)
Trades negativos: 84 (36.8%)

Retorno total: 291.85%
Retorno médio por trade: 1.280%
Desvio padrão dos trades: 3.833%
Sharpe ratio: 0.334
Drawdown máximo: -49.61%

Anomalias detectadas:
  Superiores (venda): 250
  Inferiores (compra): 255
  Total: 505
  Frequência: 22.18% dos dias


In [170]:
#Visualização
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=[
        'Delta com Bandas',
        'Sinais de Trading',
        'Performance Acumulada'
    ],
    row_heights=[0.4, 0.3, 0.3],
    vertical_spacing=0.08
)


# Delta e bandas
fig.add_trace(
    go.Scatter(x=df.index, y=df['Delta'], name='Delta', line=dict(color='purple')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df[f'banda_sup'], name='Banda Superior', 
                line=dict(color='red', dash='dash')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df[f'banda_inf'], name='Banda Inferior', 
                line=dict(color='red', dash='dash')),
    row=1, col=1
)

#Sinais de trading
compras = df[df['Regra'] == 1]
vendas = df[df['Regra'] == -1]

if not compras.empty:
    fig.add_trace(
        go.Scatter(x=compras.index, y=compras['Adj Close'], mode='markers',
                    marker=dict(color='green', size=8, symbol='triangle-up'),
                    name='Compra'),
        row=2, col=1
    )

if not vendas.empty:
    fig.add_trace(
        go.Scatter(x=vendas.index, y=vendas['Adj Close'], mode='markers',
                    marker=dict(color='red', size=8, symbol='triangle-down'),
                    name='Venda'),
        row=2, col=1
    )

fig.add_trace(
    go.Scatter(x=df.index, y=df['Adj Close'], name='Preço (ref)', 
                line=dict(color='darkblue', width=1)),
    row=2, col=1
)

#Performance acumulada
fig.add_trace(
    go.Scatter(x=df.index, y=df['Acumulado'], name='Retorno Acumulado (%)', 
                line=dict(color='darkgreen', width=2)),
    row=3, col=1
)

# Layout
fig.update_layout(
    title='Estratégia de Reversão à Média',
    height=1000,
    showlegend=True
)

fig.update_xaxes(title_text="Data", row=3, col=1)
fig.update_yaxes(title_text="Valor", row=1, col=1)
fig.update_yaxes(title_text="Preço", row=2, col=1)
fig.update_yaxes(title_text="Retorno %", row=3, col=1)

fig.show()